In [1]:
import json
import numpy as np
from cassandra.cluster import Cluster
from utils import dumpNeighborhood, Vecindario, Restaurante

## Transformación de datos de restaurantes

In [2]:
with open("./restaurants.json", "r") as file:
    with open('./restaurants.csv', 'w') as fileDump:
        fileDump.write('id,location,name,type\n')
        while True:
            line = file.readline()
            if line == '':
                break
            restaurant = json.loads(line)
            restaurant['location']['coordinates'].reverse()
            dumpLine = "{0},\"{1}\",\"{2}\",\"{3}\"\n".format(restaurant["_id"]["$oid"],
                                                   restaurant['location']['coordinates'],
                                                   restaurant["name"],
                                                   restaurant["location"]["type"])
            fileDump.write(dumpLine)

## Transformación de datos de vecinos

In [3]:
with open("./neighborhoods.json", "r") as file:
    with open("./neighborhoods.csv", "w") as dumpFile:
        dumpFile.write(
            'id, "polygonNumber", "pointNumber",name, type, point\n')
        while True:
            line = file.readline()
            if line == '':
                break
            neighborhood = json.loads(line)
            dumpNeighborhood(neighborhood, dumpFile)

## Generar información para la consulta restaurantes por vecindario

In [4]:
with open("./neighborhoods.json", "r") as file:
    i = 0
    neighborhoods = []
    while True:
        line = file.readline()
        if line == '':
            break
        obj = json.loads(line)
        neighborhoods.append(Vecindario(
            obj["_id"]["$oid"], obj['geometry']['type'], obj['geometry']['coordinates']))

In [5]:
with open("./restaurants.json", "r") as file:
    restaurants = []
    while True:
        line = file.readline()
        if line == '':
            break
        obj = json.loads(line)
        restaurants.append(Restaurante(
            obj['_id']['$oid'], obj['location']['coordinates']))

In [6]:
tuplas = []
for restaurant in restaurants:
    for neighborhood in neighborhoods:
        estaDentro = neighborhood.estaDentro(restaurant)
        if estaDentro is not None:
            tuplas.append(estaDentro)
            break

In [7]:
with open('RestaurantesPorVecindario.csv', 'w') as file:
    file.write('id_vecindario,polygonNumber,id_restaurante\n')
    for tupla in tuplas:
        file.write(f"{tupla[0]},{tupla[1]},{tupla[2]}\n")

Una vez generados los ficheros se envían al volumen en docker y se ejecutan las siguientes sentencias:
* copy restaurantsbyneighborhood (id_neighborhood,"polygonNumber",id_restaurant) from '/var/lib/cassandra/challenge/RestaurantesPorVecindario.csv' with HEADER=TRUE;

* copy neighborhoods (id, "polygonNumber","pointNumber",name,type,point) from '/var/lib/cassandra/challenge/neighborhoods.csv' with HEADER=TRUE;

* copy restaurants (id,location,name,type) from '/var/lib/cassandra/challenge/restaurants.csv' with HEADER=TRUE;